安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

In [1]:
!pip install opencompass[full]
# !pip install pytorch transformers datasets "opencompass[full]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 31.1 MB/s eta 0:00:00
  Preparing metada

# 指令微调

In [2]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""

import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets

In [ ]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
@dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here

In [ ]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path, 
        device_map = "auto",
        max_position_embeddings = 512,
        torch_dtype=getattr(torch, model_args.torch_dtype) if model_args.torch_dtype else "bfloat16"
    )

    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    dataset = datasets.load_dataset('csv', data_files=data_args.dataset_path, split='train')
    
    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    #
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!

    def data_collator(batch: List[Dict]):
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        inputs = [
            (example['instruction'] or "[NA]") + (example['input'] or "[NA]") + (example['output'] or "[NA]")  # Ensure 'None' is replaced by empty string
            for example in batch
        ]
        # targets = [example['output'] if example['output'] is not None else "[NA]" for example in batch]
        
        # Tokenize inputs and outputs
        model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length", return_tensors="pt")
        # labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
        
        # Add the labels to model_inputs
        # model_inputs['labels'] = labels['input_ids']
        model_inputs['labels'] = model_inputs['input_ids']
        # model_inputs['attention_mask'] = model_inputs['attention_mask'] + 1

        padding_value = tokenizer.pad_token_id  # or use a specific padding token value, e.g., 0
        model_inputs['labels'] = torch.where(
            model_inputs['labels'] == padding_value, 
            torch.full_like(model_inputs['labels'], fill_value=-100),  # Replace padding token with ignore_index
            model_inputs['labels']
        )

        # print("Model Input:", model_inputs)  # This will print the entire model input dictionary     
        
        # Return the processed batch, automatically padded to the same length
        return model_inputs

    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = dataset,
        tokenizer = tokenizer,
        data_collator = data_collator,
        # eval_dataset = val_dataset,
        # callbacks=[EarlyStoppingCallback(early_stopping_patience=3), LogLossCallback()],
    )

    # Step 6: Train!
    trainer.train()
    trainer.save_model("/kaggle/working/sft_model")

In [ ]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
sys.argv = [
    "notebook", 
    "--model_name_or_path", "/kaggle/input/qwen2.5/transformers/0.5b/1",
    "--dataset_path", "/kaggle/input/alpaca-language-instruction-training/train.csv",
    "--output_dir", "/kaggle/working/",
    "--logging_dir","/kaggle/working/logs",
    "--logging_steps", "40",   # 每10步输出一次训练损失
    "--logging_strategy", "steps",
    "--save_steps","4000",  # 每4000步保存一次模型检查点
    "--save_total_limit","1",  # 最多保留1个检查点，删除较旧的
    "--report_to","none",  # 禁用 W&B 集成
    "--torch_dtype" , "bfloat16",
    "--remove_unused_columns","False",#保留未使用的列
    "--per_device_train_batch_size" ,"4", # batch_size 4
    "--per_device_eval_batch_size", "4", # batch_size 4
    "--gradient_accumulation_steps", "8", # 累加向后传播次数
    "--lr_scheduler_type", "linear",  # 使用线性学习率调度器
    "--learning_rate", "5e-5", # 学习率
    "--num_train_epochs", "3", # epochs的数目
    "--adam_epsilon", "1e-8",  # 小 epsilon 提高稳定性
    "--evaluation_strategy","no",   # 每隔一定步数进行验证
    # "--load_best_model_at_end","True",   # 训练结束后加载最佳模型
    # "--eval_steps","4000",                # 每 4000 步验证一次
    # "--greater_is_better","False",  # 损失越小越好
    # "--dataloader_drop_last", "True", # 丢弃最后一个不完全的batch
    # "--metric_for_best_model","eval_loss",  # 使用验证集上的损失作为最佳模型标准
]
finetune()

# 评测模型

In [7]:
PLM_MODEL_PATH = "/kaggle/input/qwen2.5/transformers/0.5b/1"
SFT_MODEL_PATH = "/kaggle/input/mine/pytorch/default/1/sft_model"
torch.cuda.empty_cache()

如果你有多个GPU，可以修改下面的--hf-num-gpus参数来加速评测。

In [4]:
!opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {PLM_MODEL_PATH} \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "/kaggle/working/evals/plm" \
    --debug

/usr/local/lib/python3.10/dist-packages/opencompass/__init__.py:19: UserWarning: Starting from v0.4.0, all AMOTIC configuration files currently located in `./configs/datasets`, `./configs/models`, and `./configs/summarizers` will be migrated to the `opencompass/configs/` package. Please update your configuration file paths accordingly.
  _warn_about_config_migration()


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


2024-12-28 09:18:00.132902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-28 09:18:00.154425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-28 09:18:00.160934: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
12/28 09:18:04 - OpenCompass - INFO - Loading mmlu_ppl: /usr/local/lib/python3.10/dist-packages/opencompass/configs/./datasets/mmlu/mmlu_ppl.py
12/28 09:18:04 - OpenCompass - INFO - Loading hellaswag_clean_ppl: /usr/local/lib/python3.10/dist-packages/opencompass/configs/./datasets/hellaswag/hellaswag_clean_ppl.py
12/28 09:18:04 - OpenCompass - INFO - Loading wino

In [9]:
!opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {SFT_MODEL_PATH} \
    --tokenizer-kwargs padding_side="right" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "/kaggle/working/evals/sft" \
    --debug

/usr/local/lib/python3.10/dist-packages/opencompass/__init__.py:19: UserWarning: Starting from v0.4.0, all AMOTIC configuration files currently located in `./configs/datasets`, `./configs/models`, and `./configs/summarizers` will be migrated to the `opencompass/configs/` package. Please update your configuration file paths accordingly.
  _warn_about_config_migration()
2024-12-28 14:22:27.136782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-28 14:22:27.159579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-28 14:22:27.166544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regis